# IMPORTS

In [ ]:

import numpy as np
from sklearn.datasets import fetch_olivetti_faces, load_iris


# CROSS VALIDATION

In [ ]:
def get_cross_validation_data(X, Y, fold_count, fold_test):
    classes = np.unique(Y)
    class_count = len(classes)

    classes_element_ids = {}
    block_size = np.zeros(class_count, dtype=int)  # Ensure block_size is of integer type

    for class_id in range(class_count):
        classes_element_ids[class_id] = np.where(Y == classes[class_id])[0]
        
        # Calculate block size for testing (20%) and training (80%)
        block_size[class_id] = int(np.floor(len(classes_element_ids[class_id]) * 0.2)) if fold_test == 1 else int(np.floor(len(classes_element_ids[class_id]) * 0.8 / (fold_count - 1)))

    element_test_ids = np.zeros(int(np.sum(block_size)), dtype=int)
    element_train_ids = np.zeros(int(np.sum(block_size) * (fold_count - 1)), dtype=int)
    element_validate_ids = np.zeros(int(np.sum(block_size)), dtype=int)

    fold_validate = (fold_test % fold_count) + 1
    stack_counter = 0

    for class_id in range(class_count):
        test_range = np.arange(1, block_size[class_id] + 1).astype(int) + (fold_test - 1) * block_size[class_id]
        train_range = np.arange(1, block_size[class_id] * fold_count + 1).astype(int)
        train_range = np.delete(train_range, (test_range - 1).astype(int))
        validate_range = np.arange(1, block_size[class_id] + 1).astype(int) + (fold_validate - 1) * block_size[class_id]

        element_test_ids[stack_counter:stack_counter + len(test_range)] = classes_element_ids[class_id][test_range - 1]
        element_train_ids[stack_counter * (fold_count - 1):stack_counter * (fold_count - 1) + len(train_range)] = \
            classes_element_ids[class_id][train_range - 1]
        element_validate_ids[stack_counter:stack_counter + len(validate_range)] = \
            classes_element_ids[class_id][validate_range - 1]

        stack_counter += block_size[class_id]

    test = {'X': X[element_test_ids, :], 'Y': Y[element_test_ids]}
    train = {'X': X[element_train_ids, :], 'Y': Y[element_train_ids]}
    validate = {'X': X[element_validate_ids, :], 'Y': Y[element_validate_ids]}

    return test, train, validate

# LOADING DATASETS & SPLIT

In [ ]:
# Load face dataset
faces = fetch_olivetti_faces(data_home='Dataset/VISA_Face/VISA_Face')
X_faces = faces.data
y_faces = faces.target

# Load iris dataset
iris = load_iris()
X_iris = iris.data
y_iris = iris.target

# Set fold count and fold test
fold_count = 5
fold_test = 1  # Testing on the first fold

# Call the function to split the data
test_faces, train_faces, validate_faces = get_cross_validation_data(X_faces, y_faces, fold_count, fold_test)
test_iris, train_iris, validate_iris = get_cross_validation_data(X_iris, y_iris, fold_count, fold_test)

# Print the results, testing purposes
print("Face DATASET:")
print("\nTest data:", test_faces)
print("\nTrain data:", train_faces)
print("\nValidation data:", validate_faces)

print("\nIris DATASET:")
print("\nTest data:", test_iris)
print("\nTrain data:", train_iris)
print("\nValidation data:", validate_iris)